<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/metabolism_Discriptor_0821.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors

In [3]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule

        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res


In [4]:
# load data
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

train_aug_HLM = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train_aug_HLM.csv")
train_aug_MLM = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train_aug_MLM.csv")
train_IC50 = pd.read_csv("/content/drive/MyDrive/3A4_IC50.csv")[['Smiles','pChEMBL Value']]
train_IC50.columns = ['SMILES', 'pIC50']

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])



In [5]:
train_aug_HLM.columns = ['id', 'SMILES', 'HLM', 'AlogP', 'Molecular_Weight']
train_aug_MLM.columns = ['id', 'SMILES', 'MLM', 'AlogP', 'Molecular_Weight']

In [6]:
train_HLM = pd.concat([train, train_aug_HLM], axis = 0).reset_index(drop=True)
train_MLM = pd.concat([train, train_aug_MLM], axis = 0).reset_index(drop=True)

In [18]:
train_HLM

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5.0,2.0,8.0,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2.0,1.0,2.0,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5.0,0.0,3.0,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6.0,0.0,5.0,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3.0,0.0,1.0,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
4730,CHEMBL2391590,O=C(Nc1ccc(S(=O)(=O)c2ccccc2)cc1)OCc1cccnc1,NaN,75.000,3.660,368.410,NaN,NaN,NaN,NaN,NaN
4731,CHEMBL2391579,O=C(NCc1ccccc1)Nc1ccc(S(=O)(=O)N2CCCCC2)cc1,NaN,75.000,3.180,373.480,NaN,NaN,NaN,NaN,NaN
4732,CHEMBL2391585,O=C(NCc1ccc2nccn2c1)Nc1ccc(S(=O)(=O)c2ccccc2)cc1,NaN,49.000,3.490,406.470,NaN,NaN,NaN,NaN,NaN
4733,CHEMBL2391571,O=C(NCc1cccnc1)Nc1cccc(S(=O)(=O)N2CCCCC2)c1,NaN,40.000,2.580,374.470,NaN,NaN,NaN,NaN,NaN


In [17]:
train_MLM

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5.0,2.0,8.0,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2.0,1.0,2.0,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5.0,0.0,3.0,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6.0,0.0,5.0,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3.0,0.0,1.0,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
4209,CHEMBL2401971,CC[C@@H]1C(=O)N(C)c2cnc(-n3ccnc3-c3ccccc3)nc2N...,13.000,NaN,2.680,404.470,NaN,NaN,NaN,NaN,NaN
4210,CHEMBL112,CC(=O)Nc1ccc(O)cc1,65.000,NaN,1.350,151.160,NaN,NaN,NaN,NaN,NaN
4211,CHEMBL2414787,CC(C)N[C@@H](CC(C)(C)C)C(=O)N[C@H]1CC[C@@H]2CN...,64.300,NaN,4.020,503.630,NaN,NaN,NaN,NaN,NaN
4212,CHEMBL2414776,CC(C)C[C@H](N)C(=O)N[C@@H]1CC[C@H]2CN(S(=O)(=O...,10.300,NaN,2.590,447.520,NaN,NaN,NaN,NaN,NaN


In [25]:
ic50_prediction = pd.read_csv('/content/drive/MyDrive/metabolism_dacon/ic50_prediction.csv')
ic50_prediction = ic50_prediction.dropna(subset=['id'])
ic50_prediction = ic50_prediction.drop_duplicates(subset=['id'])

In [26]:
duplicates = ic50_prediction[ic50_prediction.duplicated('id')]
print(duplicates)

Empty DataFrame
Columns: [index, SMILES, IC50, label, id, MLM, HLM, AlogP, Molecular_Weight, Num_H_Acceptors, Num_H_Donors, Num_RotatableBonds, LogD, Molecular_PolarSurfaceArea, Molecule]
Index: []


In [27]:
ic50_prediction

,index,SMILES,IC50,label,id,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
5742,0,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,5.184304,train,TRAIN_0000,26.010,50.680,3.259,400.495,5.0,2.0,8.0,3.259,117.37,<rdkit.Chem.rdchem.Mol object at 0x7bc8470668f0>
5743,1,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,4.994711,train,TRAIN_0001,29.270,50.590,2.169,301.407,2.0,1.0,2.0,2.172,73.47,<rdkit.Chem.rdchem.Mol object at 0x7bc847066960>
5744,2,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,4.902590,train,TRAIN_0002,5.586,80.892,1.593,297.358,5.0,0.0,3.0,1.585,62.45,<rdkit.Chem.rdchem.Mol object at 0x7bc8470669d0>
5745,3,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.138790,train,TRAIN_0003,5.710,2.000,4.771,494.652,6.0,0.0,5.0,3.475,92.60,<rdkit.Chem.rdchem.Mol object at 0x7bc847066a40>
5746,4,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,5.142610,train,TRAIN_0004,93.270,99.990,2.335,268.310,3.0,0.0,1.0,2.337,42.43,<rdkit.Chem.rdchem.Mol object at 0x7bc847066ab0>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11671,478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,4.878050,test,TEST_478,NaN,NaN,4.207,306.443,2.0,1.0,7.0,4.207,55.13,<rdkit.Chem.rdchem.Mol object at 0x7bc882895e00>
11672,479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,4.757570,test,TEST_479,NaN,NaN,-0.608,335.398,5.0,0.0,1.0,-1.736,70.16,<rdkit.Chem.rdchem.Mol object at 0x7bc882895d90>
11673,480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,4.959922,test,TEST_480,NaN,NaN,1.792,349.383,3.0,1.0,3.0,1.792,69.72,<rdkit.Chem.rdchem.Mol object at 0x7bc882895d20>
11674,481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,4.907931,test,TEST_481,NaN,NaN,0.790,341.132,3.0,2.0,2.0,0.423,69.64,<rdkit.Chem.rdchem.Mol object at 0x7bc882895cb0>


In [28]:
full_HLM = pd.concat([train_HLM, test], axis = 0).reset_index(drop=True)
full_HLM = pd.merge(left=full_HLM, right=ic50_prediction[['id', 'IC50']], how='inner')
full_HLM['Molecule'] = full_HLM['SMILES'].apply(Chem.MolFromSmiles)

full_MLM = pd.concat([train_MLM, test], axis = 0).reset_index(drop=True)
full_MLM = pd.merge(left=full_MLM, right=ic50_prediction[['id', 'IC50']], how='inner')
full_MLM['Molecule'] = full_MLM['SMILES'].apply(Chem.MolFromSmiles)

In [29]:
full_HLM

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,IC50,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5.0,2.0,8.0,3.259,117.37,5.184304,<rdkit.Chem.rdchem.Mol object at 0x7b318e9945f0>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2.0,1.0,2.0,2.172,73.47,4.994711,<rdkit.Chem.rdchem.Mol object at 0x7b318e994580>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5.0,0.0,3.0,1.585,62.45,4.902590,<rdkit.Chem.rdchem.Mol object at 0x7b318e994660>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6.0,0.0,5.0,3.475,92.60,5.138790,<rdkit.Chem.rdchem.Mol object at 0x7b318e997ae0>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3.0,0.0,1.0,2.337,42.43,5.142610,<rdkit.Chem.rdchem.Mol object at 0x7b318e994c10>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5213,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,4.207,306.443,2.0,1.0,7.0,4.207,55.13,4.878050,<rdkit.Chem.rdchem.Mol object at 0x7b318e6c9070>
5214,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,-0.608,335.398,5.0,0.0,1.0,-1.736,70.16,4.757570,<rdkit.Chem.rdchem.Mol object at 0x7b318e6c9000>
5215,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,1.792,349.383,3.0,1.0,3.0,1.792,69.72,4.959922,<rdkit.Chem.rdchem.Mol object at 0x7b318e6c8f90>
5216,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,0.790,341.132,3.0,2.0,2.0,0.423,69.64,4.907931,<rdkit.Chem.rdchem.Mol object at 0x7b318e6c8f20>


In [30]:
full_MLM

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,IC50,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5.0,2.0,8.0,3.259,117.37,5.184304,<rdkit.Chem.rdchem.Mol object at 0x7b318f99f920>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2.0,1.0,2.0,2.172,73.47,4.994711,<rdkit.Chem.rdchem.Mol object at 0x7b318e6c8e40>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5.0,0.0,3.0,1.585,62.45,4.902590,<rdkit.Chem.rdchem.Mol object at 0x7b318e6c8dd0>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6.0,0.0,5.0,3.475,92.60,5.138790,<rdkit.Chem.rdchem.Mol object at 0x7b318e6c8d60>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3.0,0.0,1.0,2.337,42.43,5.142610,<rdkit.Chem.rdchem.Mol object at 0x7b318e6c8cf0>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4692,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,4.207,306.443,2.0,1.0,7.0,4.207,55.13,4.878050,<rdkit.Chem.rdchem.Mol object at 0x7b318e87b760>
4693,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,-0.608,335.398,5.0,0.0,1.0,-1.736,70.16,4.757570,<rdkit.Chem.rdchem.Mol object at 0x7b318e87b6f0>
4694,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,1.792,349.383,3.0,1.0,3.0,1.792,69.72,4.959922,<rdkit.Chem.rdchem.Mol object at 0x7b318e87b680>
4695,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,0.790,341.132,3.0,2.0,2.0,0.423,69.64,4.907931,<rdkit.Chem.rdchem.Mol object at 0x7b318e87b610>


In [31]:
from rdkit import DataStructs

allDescrs_HLM = [getMolDescriptors(m) for m in full_HLM['Molecule']]
full_Descrs_HLM = pd.DataFrame(allDescrs_HLM)

allDescrs_MLM = [getMolDescriptors(m) for m in full_MLM['Molecule']]
full_Descrs_MLM = pd.DataFrame(allDescrs_MLM)

In [32]:
Data_HLM = pd.concat([full_HLM, full_Descrs_HLM], axis=1)
Data_HLM['Num_H_Acceptors'] = Data_HLM['Num_H_Acceptors'].fillna(Data_HLM['NumHAcceptors'])
Data_HLM['Num_H_Donors'] = Data_HLM['Num_H_Donors'].fillna(Data_HLM['NumHDonors'])
Data_HLM['Num_RotatableBonds'] = Data_HLM['Num_RotatableBonds'].fillna(Data_HLM['NumRotatableBonds'])
Data_HLM['LogD'] = Data_HLM['LogD'].fillna(Data_HLM['MolLogP'])
Data_HLM['Molecular_PolarSurfaceArea'] = Data_HLM['Molecular_PolarSurfaceArea'].fillna(Data_HLM['TPSA'])

Data_MLM = pd.concat([full_MLM, full_Descrs_MLM], axis=1)
Data_MLM['Num_H_Acceptors'] = Data_MLM['Num_H_Acceptors'].fillna(Data_MLM['NumHAcceptors'])
Data_MLM['Num_H_Donors'] = Data_MLM['Num_H_Donors'].fillna(Data_MLM['NumHDonors'])
Data_MLM['Num_RotatableBonds'] = Data_MLM['Num_RotatableBonds'].fillna(Data_MLM['NumRotatableBonds'])
Data_MLM['LogD'] = Data_MLM['LogD'].fillna(Data_MLM['MolLogP'])
Data_MLM['Molecular_PolarSurfaceArea'] = Data_MLM['Molecular_PolarSurfaceArea'].fillna(Data_MLM['TPSA'])

In [35]:
for col in Data_HLM.columns:
    has_missing_values = Data_HLM[col].isna().any()
    count =  Data_HLM[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

for col in Data_MLM.columns:
    has_missing_values = Data_MLM[col].isna().any()
    count =  Data_MLM[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

Column 'MLM' has missing values.
'1720'
Column 'HLM' has missing values.
'483'
Column 'MaxPartialCharge' has missing values.
'2'
Column 'MinPartialCharge' has missing values.
'2'
Column 'MaxAbsPartialCharge' has missing values.
'2'
Column 'MinAbsPartialCharge' has missing values.
'2'
Column 'BCUT2D_MWHI' has missing values.
'3'
Column 'BCUT2D_MWLOW' has missing values.
'3'
Column 'BCUT2D_CHGHI' has missing values.
'3'
Column 'BCUT2D_CHGLO' has missing values.
'3'
Column 'BCUT2D_LOGPHI' has missing values.
'3'
Column 'BCUT2D_LOGPLOW' has missing values.
'3'
Column 'BCUT2D_MRHI' has missing values.
'3'
Column 'BCUT2D_MRLOW' has missing values.
'3'
Column 'MLM' has missing values.
'483'
Column 'HLM' has missing values.
'1199'
Column 'MaxPartialCharge' has missing values.
'2'
Column 'MinPartialCharge' has missing values.
'2'
Column 'MaxAbsPartialCharge' has missing values.
'2'
Column 'MinAbsPartialCharge' has missing values.
'2'
Column 'BCUT2D_MWHI' has missing values.
'3'
Column 'BCUT2D_M

In [36]:
from rdkit.Chem import AllChem
fp_list = []
for idx, smiles in enumerate(full_HLM["SMILES"]): # Smiles 열에서 반복.
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits = 4096)
    fp_list.append(fp)

import sys
fp_bit_list = []
for fp in fp_list:
  fp_bit_list.append([int(x) for x in fp.ToBitString()])

dic = {}
for idx in range(4096):
  key = "F" + str(idx)
  dic[key] = []

for fp in fp_bit_list:
  for idx in range(4096):
    key = "F" + str(idx)
    dic[key].append(fp[idx])

Data_fp_HLM = pd.DataFrame(dic)

In [38]:
from rdkit.Chem import AllChem
fp_list = []
for idx, smiles in enumerate(full_MLM["SMILES"]): # Smiles 열에서 반복.
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits = 4096)
    fp_list.append(fp)

import sys
fp_bit_list = []
for fp in fp_list:
  fp_bit_list.append([int(x) for x in fp.ToBitString()])

dic = {}
for idx in range(4096):
  key = "F" + str(idx)
  dic[key] = []

for fp in fp_bit_list:
  for idx in range(4096):
    key = "F" + str(idx)
    dic[key].append(fp[idx])

Data_fp_MLM = pd.DataFrame(dic)

In [39]:
Data_HLM = Data_HLM.drop(columns=['AlogP', 'Molecule', 'MolWt', 'MLM', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

missing_indices = Data_HLM[:4735][Data_HLM[:4735].isnull().any(axis=1)].index

train_HLM = Data_HLM.iloc[:4735].dropna(axis=0)
train_fp_HLM = Data_fp_HLM.iloc[:4735].drop(missing_indices)
test_HLM = Data_HLM.iloc[4735:]
test_fp_HLM = Data_fp_HLM.iloc[4735:]

In [40]:
train_HLM

,id,SMILES,HLM,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,IC50,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,50.680,400.495,5.0,2.0,8.0,3.2590,117.37,5.184304,...,0,0,0,0,0,1,0,0,0,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,50.590,301.407,2.0,1.0,2.0,2.1720,73.47,4.994711,...,0,0,0,0,0,1,0,0,0,0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,80.892,297.358,5.0,0.0,3.0,1.5850,62.45,4.902590,...,0,0,0,0,1,0,0,0,0,0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,2.000,494.652,6.0,0.0,5.0,3.4750,92.60,5.138790,...,0,1,0,0,0,0,0,0,0,0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,99.990,268.310,3.0,0.0,1.0,2.3370,42.43,5.142610,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730,CHEMBL2391587,O=C(NCc1ccc(-n2ccnc2)nc1)Nc1ccc(S(=O)(=O)c2ccc...,77.000,433.490,6.0,2.0,6.0,3.4218,105.98,5.677434,...,0,0,1,0,0,0,0,0,0,1
4731,CHEMBL2391590,O=C(Nc1ccc(S(=O)(=O)c2ccccc2)cc1)OCc1cccnc1,75.000,368.410,5.0,1.0,5.0,3.6631,85.36,5.784332,...,0,0,1,0,0,0,0,0,0,0
4732,CHEMBL2391579,O=C(NCc1ccccc1)Nc1ccc(S(=O)(=O)N2CCCCC2)cc1,75.000,373.480,3.0,2.0,5.0,3.1829,78.51,5.230156,...,0,1,0,0,0,0,0,0,0,1
4733,CHEMBL2391585,O=C(NCc1ccc2nccn2c1)Nc1ccc(S(=O)(=O)c2ccccc2)cc1,49.000,406.470,5.0,2.0,5.0,3.4888,92.57,5.133878,...,0,0,1,0,0,0,0,0,0,1


In [41]:
Data_MLM = Data_MLM.drop(columns=['AlogP', 'Molecule', 'MolWt', 'HLM', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

missing_indices = Data_MLM.iloc[:4214][Data_MLM.iloc[:4214].isnull().any(axis=1)].index

train_MLM = Data_MLM.iloc[:4214].dropna(axis=0)
train_fp_MLM = Data_fp_MLM.iloc[:4214].drop(missing_indices)
test_MLM = Data_MLM.iloc[4214:]
test_fp_MLM = Data_fp_MLM.iloc[4214:]

In [42]:
train_MLM

,id,SMILES,MLM,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,IC50,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,400.495,5.0,2.0,8.0,3.2590,117.37,5.184304,...,0,0,0,0,0,1,0,0,0,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,301.407,2.0,1.0,2.0,2.1720,73.47,4.994711,...,0,0,0,0,0,1,0,0,0,0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,297.358,5.0,0.0,3.0,1.5850,62.45,4.902590,...,0,0,0,0,1,0,0,0,0,0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,494.652,6.0,0.0,5.0,3.4750,92.60,5.138790,...,0,1,0,0,0,0,0,0,0,0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,268.310,3.0,0.0,1.0,2.3370,42.43,5.142610,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4209,CHEMBL2401971,CC[C@@H]1C(=O)N(C)c2cnc(-n3ccnc3-c3ccccc3)nc2N...,13.000,404.470,7.0,0.0,4.0,2.6796,76.38,5.489698,...,0,0,0,0,0,0,0,0,0,0
4210,CHEMBL112,CC(=O)Nc1ccc(O)cc1,65.000,151.160,2.0,2.0,1.0,1.3506,49.33,5.036601,...,0,0,0,0,0,0,0,0,0,0
4211,CHEMBL2414787,CC(C)N[C@@H](CC(C)(C)C)C(=O)N[C@H]1CC[C@@H]2CN...,64.300,503.630,4.0,2.0,7.0,4.0235,78.51,4.770013,...,0,1,0,0,0,0,0,0,0,0
4212,CHEMBL2414776,CC(C)C[C@H](N)C(=O)N[C@@H]1CC[C@H]2CN(S(=O)(=O...,10.300,447.520,4.0,2.0,6.0,2.5941,92.50,4.994215,...,0,1,0,0,0,0,0,0,0,0


In [43]:
# 숫자형식 컬럼들의 min-max 정규화
scaler = MinMaxScaler()
non_features = ['id', 'SMILES', 'HLM','Fingerprint']
features = [column for column in train_HLM.columns if column not in non_features]
train_HLM[features] = scaler.fit_transform(train_HLM[features])

test_HLM[features] = scaler.transform(test_HLM[features])

<ipython-input-43-6b907aab1527>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_HLM[features] = scaler.transform(test_HLM[features])


In [44]:
scaler = MinMaxScaler()
non_features = ['id', 'SMILES', 'MLM','Fingerprint']
features = [column for column in train_MLM.columns if column not in non_features]
train_MLM[features] = scaler.fit_transform(train_MLM[features])

test_MLM[features] = scaler.transform(test_MLM[features])

<ipython-input-44-d1c53a1fb12e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_MLM[features] = scaler.transform(test_MLM[features])


In [85]:
numeric_columns = train_MLM.select_dtypes(include='number')
correlation_matrix = numeric_columns.corr()

sorted_corr_mlm = correlation_matrix['MLM'].sort_values(ascending=False).dropna()

positive_top_20 = sorted_corr_mlm[sorted_corr_mlm > 0].head(20)
negative_top_20 = sorted_corr_mlm[sorted_corr_mlm < 0].tail(20)

top_20_columns = positive_top_20.index.tolist() + negative_top_20.index.tolist()

print("Positive Correlations - Top 20:")
print(positive_top_20)

print("\nNegative Correlations - Top 20:")
print(negative_top_20)

Positive Correlations - Top 20:
MLM                           1.000000
TPSA                          0.241668
fr_COO                        0.229694
fr_COO2                       0.229694
Num_H_Donors                  0.219859
NHOHCount                     0.217183
EState_VSA10                  0.187358
Molecular_PolarSurfaceArea    0.180222
fr_Al_COO                     0.180191
VSA_EState3                   0.177790
NOCount                       0.171339
NumHeteroatoms                0.167678
MaxPartialCharge              0.153999
Num_H_Acceptors               0.151213
MinAbsPartialCharge           0.150298
fr_Ar_COO                     0.147125
FpDensityMorgan1              0.142220
VSA_EState2                   0.131641
PEOE_VSA14                    0.122064
fr_NH1                        0.118692
Name: MLM, dtype: float64

Negative Correlations - Top 20:
MinAbsEStateIndex        -0.085756
Chi4v                    -0.087724
MolMR                    -0.088454
VSA_EState8             

In [104]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint', 'AlogP']
features = [column for column in top_20_columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train_MLM)):
    df_train = train_MLM.iloc[train_index]
    df_valid = train_MLM.iloc[valid_index]

    df_train_fp = train_fp_MLM.iloc[train_index]
    df_valid_fp = train_fp_MLM.iloc[valid_index]

    x_train_num = df_train[features].values
    x_train_fingerprint =  df_train_fp.values  # Extract fingerprints

    y_mlm_train = df_train[mlm_target].values

    x_valid_num = df_valid[features].values
    x_valid_fingerprint = df_valid_fp.values  # Extract fingerprints

    y_mlm_valid = df_valid[mlm_target].values

    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(128, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    x1 = layers.Dense(256, activation='relu')(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    x1 = layers.Dense(512, activation='relu')(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)

    # # 비트 입력 경로
    x2_input = keras.Input(shape=(x_train_fingerprint.shape[1],))
    x2 = layers.Flatten()(x2_input)
    x2 = layers.Dropout(0.4)(x2)

    # 경로 병합
    merged = layers.concatenate([x1, x2])
    merged = layers.Dense(1024, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model = keras.Model(inputs=[x1_input, x2_input], outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0008,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])


    # Train the model
    checkpoint_mlm = ModelCheckpoint(f'model_mlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_mlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model.fit([x_train_num, x_train_fingerprint], y_mlm_train, epochs=500, batch_size=32, verbose=1, validation_data=([x_valid_num, x_valid_fingerprint], y_mlm_valid),
              callbacks=[checkpoint_mlm, early_stopping_mlm])

    reg_mlms.append(model)

    # Calculate RMSE for MLM predictions
    y_mlm_pred = model.predict([x_valid_num, x_valid_fingerprint])
    mlm_rmse = math.sqrt(mean_squared_error(y_mlm_valid, y_mlm_pred))
    print(mlm_rmse)
    mlm_rmse_scores.append(mlm_rmse)

Epoch 1/500
119/119 [==============================] - 3s 11ms/step - loss: 1471.3983 - root_mean_squared_error: 38.3588 - val_loss: 1138.4912 - val_root_mean_squared_error: 33.7415
Epoch 2/500
119/119 [==============================] - 1s 8ms/step - loss: 1041.7644 - root_mean_squared_error: 32.2764 - val_loss: 1039.3114 - val_root_mean_squared_error: 32.2384
Epoch 3/500
119/119 [==============================] - 1s 8ms/step - loss: 995.4240 - root_mean_squared_error: 31.5503 - val_loss: 1007.2095 - val_root_mean_squared_error: 31.7366
Epoch 4/500
119/119 [==============================] - 1s 8ms/step - loss: 980.8159 - root_mean_squared_error: 31.3180 - val_loss: 987.2390 - val_root_mean_squared_error: 31.4204
Epoch 5/500
119/119 [==============================] - 1s 8ms/step - loss: 954.7938 - root_mean_squared_error: 30.8997 - val_loss: 930.9032 - val_root_mean_squared_error: 30.5107
Epoch 6/500
119/119 [==============================] - 1s 7ms/step - loss: 919.0800 - root_mean_squ

In [76]:
correlation_matrix

,MLM,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,IC50,MaxAbsEStateIndex,MaxEStateIndex,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
MLM,1.000000,-0.022632,0.151213,0.219859,-0.034163,-0.247690,0.180222,-0.156479,0.086909,0.086909,...,0.005836,0.006716,0.008712,NaN,0.008077,-0.032763,NaN,-0.047834,0.044925,0.037256
Molecular_Weight,-0.022632,1.000000,0.487420,0.099981,0.569885,0.477953,0.395479,0.247871,0.362327,0.362327,...,-0.016344,0.228494,0.106422,NaN,-0.033887,-0.042542,NaN,0.079056,0.109859,0.049836
Num_H_Acceptors,0.151213,0.487420,1.000000,0.174021,0.439282,-0.176029,0.673698,0.055982,0.161089,0.161089,...,0.090153,0.116323,0.156389,NaN,0.077987,0.016949,NaN,-0.006411,0.083449,-0.098354
Num_H_Donors,0.219859,0.099981,0.174021,1.000000,0.177533,-0.177941,0.469775,-0.017376,0.096926,0.096926,...,-0.014659,0.037652,-0.063089,NaN,0.004230,-0.015153,NaN,-0.007058,0.073957,0.149385
Num_RotatableBonds,-0.034163,0.569885,0.439282,0.177533,1.000000,0.169378,0.351133,0.075669,0.182875,0.182875,...,-0.000129,0.152449,0.074771,NaN,0.003169,-0.039301,NaN,-0.041110,0.452004,-0.020494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fr_thiazole,-0.032763,-0.042542,0.016949,-0.015153,-0.039301,-0.016926,0.172240,0.049786,-0.084691,-0.084691,...,0.007265,-0.015346,0.019693,NaN,-0.026015,1.000000,NaN,-0.035311,-0.031310,-0.026141
fr_thiocyan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fr_thiophene,-0.047834,0.079056,-0.006411,-0.007058,-0.041110,0.095993,0.218195,0.014748,0.036718,0.036718,...,-0.010960,0.048918,0.040219,NaN,-0.012456,-0.035311,NaN,1.000000,-0.053557,0.095887
fr_unbrch_alkane,0.044925,0.109859,0.083449,0.073957,0.452004,0.089626,0.044781,-0.091200,0.004253,0.004253,...,-0.017289,-0.023879,-0.034120,NaN,0.003571,-0.031310,NaN,-0.053557,1.000000,0.011085


In [72]:
train_MLM['IC50']

0       0.387229
1       0.303734
2       0.263164
3       0.367185
4       0.368868
          ...   
4209    0.521723
4210    0.322182
4211    0.204777
4212    0.303515
4213    0.444106
Name: IC50, Length: 4211, dtype: float64

In [105]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint', 'AlogP']
features = [column for column in top_20_columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train_HLM)):
    df_train = train_HLM.iloc[train_index]
    df_valid = train_HLM.iloc[valid_index]

    df_train_fp = train_fp_HLM.iloc[train_index]
    df_valid_fp = train_fp_HLM.iloc[valid_index]

    x_train_num = df_train[features].values
    x_train_fingerprint =  df_train_fp.values  # Extract fingerprints

    y_hlm_train = df_train[hlm_target].values

    x_valid_num = df_valid[features].values
    x_valid_fingerprint = df_valid_fp.values  # Extract fingerprints

    y_hlm_valid = df_valid[hlm_target].values

    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(128, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    x1 = layers.Dense(256, activation='relu')(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    x1 = layers.Dense(512, activation='relu')(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)

    # # 비트 입력 경로
    x2_input = keras.Input(shape=(x_train_fingerprint.shape[1],))
    x2 = layers.Flatten()(x2_input)
    x2 = layers.Dropout(0.4)(x2)

    # 경로 병합
    merged = layers.concatenate([x1, x2])
    merged = layers.Dense(1024, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model_hlm = keras.Model(inputs=[x1_input, x2_input], outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0008,
    decay_steps=4,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model_hlm.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model_hlm.fit([x_train_num, x_train_fingerprint], y_hlm_train, epochs=500, batch_size=32, verbose=1, validation_data=([x_valid_num, x_valid_fingerprint], y_hlm_valid),
                  callbacks=[checkpoint_hlm, early_stopping_hlm])

    reg_hlms.append(model_hlm)

    # Calculate RMSE for HLM predictions
    y_hlm_pred = model_hlm.predict([x_valid_num, x_valid_fingerprint])
    hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    print(hlm_rmse)
    hlm_rmse_scores.append(hlm_rmse)

Epoch 1/500
134/134 [==============================] - 3s 10ms/step - loss: 1651.1794 - root_mean_squared_error: 40.6347 - val_loss: 34405448.0000 - val_root_mean_squared_error: 5865.6157
Epoch 2/500
134/134 [==============================] - 1s 8ms/step - loss: 1147.0439 - root_mean_squared_error: 33.8680 - val_loss: 1121.7334 - val_root_mean_squared_error: 33.4923
Epoch 3/500
134/134 [==============================] - 1s 7ms/step - loss: 1093.8842 - root_mean_squared_error: 33.0739 - val_loss: 1482.3311 - val_root_mean_squared_error: 38.5011
Epoch 4/500
134/134 [==============================] - 1s 7ms/step - loss: 1075.9561 - root_mean_squared_error: 32.8018 - val_loss: 1285.9636 - val_root_mean_squared_error: 35.8603
Epoch 5/500
134/134 [==============================] - 1s 8ms/step - loss: 1068.3618 - root_mean_squared_error: 32.6858 - val_loss: 954.3312 - val_root_mean_squared_error: 30.8923
Epoch 6/500
134/134 [==============================] - 1s 7ms/step - loss: 1050.6844 - ro

In [110]:
from keras.models import load_model

# Load MLM models
mlm_models = [load_model(f'model_mlm_fold{i}.h5') for i in range(n_splits)]

# Load HLM models
hlm_models = [load_model(f'model_hlm_fold{i}.h5') for i in range(n_splits)]


x_test_MLM = test_MLM[features].values
x_test_fp_MLM = test_fp_MLM.values

x_test_HLM = test_HLM[features].values
x_test_fp_HLM = test_fp_HLM.values

mlm_predictions = []
hlm_predictions = []

for model_mlm, model_hlm in zip(mlm_models, hlm_models):
    mlm_predictions.append(model_mlm.predict([x_test_MLM, x_test_fp_MLM]))
    hlm_predictions.append(model_hlm.predict([x_test_HLM, x_test_fp_HLM]))

# Convert prediction lists to numpy arrays
mlm_predictions = np.array(mlm_predictions)
hlm_predictions = np.array(hlm_predictions)

mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
hlm_ensemble_prediction = hlm_predictions.mean(axis=0)

16/16 [==============================] - 0s 2ms/step


In [111]:
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = mlm_ensemble_prediction
df_submission["HLM"] = hlm_ensemble_prediction
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

In [112]:
df_submission

,id,MLM,HLM
0,TEST_000,36.342682,62.349224
1,TEST_001,67.275764,70.417648
2,TEST_002,36.200432,54.909706
3,TEST_003,47.811584,62.743359
4,TEST_004,62.307758,70.874039
...,...,...,...
478,TEST_478,13.485441,32.010593
479,TEST_479,75.705246,82.903564
480,TEST_480,39.919640,57.344093
481,TEST_481,63.384804,76.713997
